# Uraniborg with preprocessed data
![](https://ventrafiken.se/wp-content/uploads/2020/02/ura-20130125-655x311-1.png.webp)

I have added a kedro pipeline to do the preprocessing of data:
* **Join Files Node** : join all data from all files.
* **Preprocess Node**
* **Preprocess Trip Columns Node**

Some trip statistics is also calculated.
You will need to run this pipeline to get the new datasets.

* Some new dependencies are needed which can be installed with: In the project folder type: ```kedro install```
* Then type: ```kedro run``` this should do the preprocessing and store the preprocessed data. How to access these new datasets is shown below.

![](pipeline.PNG)

In [ ]:
%load_ext autoreload
%autoreload 2

from d2e2f.visualization.visualize import plot_map, plot_trips
import pandas as pd

## Loading the data

In [ ]:
%reload_kedro
#loaders = catalog.load(f'uraniborg.raw_data')

df = catalog.load('uraniborg.data_with_trip_columns')
df = df.iloc[-10000:].copy()  # Taking just a few samples for demo

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
df['Engine load ME1 (%)']*1397

In [ ]:
df.longitude.quantile(0.10)

In [ ]:
df.longitude.quantile(0.90)

And here is a nice map of the data. You may have to install "folium" to get this to work:

```pip install folium```

In [ ]:
plot_trips(df=df, zoom_start=13)

## One trip

In [ ]:
trip = df.groupby('trip_no').get_group(1)
trip.head()

In [ ]:
plot_map(trip)

## Statistics

In [ ]:
df_statistics = catalog.load("uraniborg.trip_statistics_clean")

In [ ]:
df_statistics.describe()

In [ ]:
df_statistics.hist(column='sog', bins=100)

In [ ]:
import re
regexp = re.compile(r'P\d+$')
if regexp.search('P10g0'):
    print('yes')